In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import stock_module as sm
import random
import os
import datetime
import time
import openpyxl
from sklearn.preprocessing import scale
from scipy.signal import savgol_filter

In [4]:
def remove_noise(frame, first_par, second_par):
    w_out_noise = savgol_filter(frame["StockValue"], first_par, second_par)
    frame["StockValue"] = w_out_noise
    frame_f = frame[["StockValue"]]
    return frame_f

In [5]:
def ma_columns(frame_column, list_ma):
    MAs = [] 
    for ma in list_ma:
        MA = pd.DataFrame(sm.short_moving_average(frame_column, ma)["StockValue"]).rename(columns={"StockValue": f"MA_{ma}"})
        MAs.append(MA)
        
    frame = MAs[0].join(MAs[1])
    for i in range(len(MAs)-2):
        frame = frame.join(MAs[i+2])
    
    frame = frame.dropna()
    
    return frame

In [101]:
def buy_sell_recommendations(new_frame):
    new_frame['rec_1'] = np.where(((new_frame['MA_7_50']>=0) & (new_frame['MA_7_50']*1 > new_frame["MA_50_200"]*0.8))
                                  | ((new_frame['MA_7_50']<0) & (new_frame['MA_7_50']*0.8 > new_frame["MA_50_200"]*1))
                                         ,"BUY", "SELL")
    
    new_frame['rec_2'] = np.where(((new_frame['MA_14_100']>=0) & (new_frame['MA_14_100']*1 > new_frame["MA_50_200"]*0.8))
                              | ((new_frame['MA_14_100']<0) & (new_frame['MA_14_100']*0.8 > new_frame["MA_50_200"]*1))
                                     ,"BUY", "SELL")
    
    return new_frame

In [7]:
def get_locations(new_frame):
    loc_first=[]
    for row in range(new_frame.shape[0]-1):

            if new_frame.iloc[row,:].rec_1 == "SELL":
                if new_frame.iloc[row+1,:].rec_1 == "BUY":
                    start = row+1
                    loc_first.append(start)

            if new_frame.iloc[row,:].rec_1 == "BUY":
                if new_frame.iloc[row+1,:].rec_1 == "SELL":
                    end = row+2
                    loc_first.append(end)
                    
    loc_second=[]
    for row in range(new_frame.shape[0]-1):

        if new_frame.iloc[row,:].rec_2 == "SELL":
            if new_frame.iloc[row+1,:].rec_2 == "BUY":
                start = row+1
                loc_second.append(start)

        if new_frame.iloc[row,:].rec_2 == "BUY":
            if new_frame.iloc[row+1,:].rec_2 == "SELL":
                end = row+2
                loc_second.append(end)
                
    return [loc_first, loc_second]

In [8]:
def get_transactions(new_frame, column, locations):
    if new_frame.iloc[0,:][str(column)] == "SELL":
        transactions = [[locations[buy], locations[sell]] for buy, sell in zip(range(0,len(locations),2), range(1,len(locations),2))]
    else:
        transactions = [[locations[buy], locations[sell]] for buy, sell in zip(range(1,len(locations),2), range(2,len(locations),2))]
        transactions.insert(0, [0, locations[0]])

    return transactions

In [9]:
def returns(new_frame, list_of_transactions):
    returns = []
    for transactions in list_of_transactions:
        buy = list(new_frame.iloc[transactions[0]:transactions[1],:]["StockValue"])[0]
        sell = list(new_frame.iloc[transactions[0]:transactions[1],:]["StockValue"])[-1]
        ret = (sell-buy)/buy
        returns.append(ret)
    
    return returns

In [10]:
def prod(list_of_all_returns):
    return np.prod([ret+1 for ret in list_of_all_returns]) 

In [11]:
def get_only_frame(stock_name, start_date, end_date, ma_list, noise_first_par, noise_second_par):
    frame = sm.extract_stock_values(start_date, end_date, [stock_name])
    frame = frame.drop_duplicates(subset=["HGDG_HS_KODU", "HGDG_TARIH"], keep="last")
    frame = frame.rename(columns={"HGDG_KAPANIS": "StockValue"})
    frame = frame.sort_values(by=["HGDG_TARIH"])
    frame = frame.reset_index(drop=True)
    frame.to_pickle(stock_name)
    
    frame_f = remove_noise(frame, noise_first_par, noise_second_par)
    new_frame = ma_columns(frame_f, ma_list)
    
    new_frame["MA_7_50"] = new_frame["MA_7"] - new_frame["MA_50"]
    new_frame["MA_14_100"] = new_frame["MA_14"] - new_frame["MA_100"]
    new_frame["MA_50_200"] = new_frame["MA_50"] - new_frame["MA_200"]
    
    frame = pd.read_pickle(stock_name)
    frame = frame[["HGDG_TARIH", "StockValue"]]
    new_frame = new_frame.join(frame)
    
    new_frame = buy_sell_recommendations(new_frame)
    new_frame = new_frame.rename(columns={"HGDG_TARIH": "Date"})
    
    return new_frame

In [12]:
def convert_date(date):
    return datetime.datetime.strptime(date, "%d-%m-%Y")

In [13]:
def convert_str_from_date(date: str):
    return datetime.datetime.strftime(date, "%d-%m-%Y")

In [14]:
def increment_date(date:str, increment_day:int):
    date = convert_date(date)
    date = date + datetime.timedelta(days=increment_day)
    date = convert_str_from_date(date)
    
    return date

In [15]:
increment_date("01-01-2015", 2)

'03-01-2015'

In [16]:
def simulation(stock_name, start_date, end_date, ma_list, noise_first_par, noise_second_par):
    
    frame = sm.extract_stock_values(start_date, end_date, [stock_name])
    frame = frame.drop_duplicates(subset=["HGDG_HS_KODU", "HGDG_TARIH"], keep="last")
    frame = frame.rename(columns={"HGDG_KAPANIS": "StockValue"})
    frame = frame.sort_values(by=["HGDG_TARIH"])
    frame = frame.reset_index(drop=True)
    frame.to_pickle(stock_name)
    
    frame_f = remove_noise(frame, noise_first_par, noise_second_par)
    new_frame = ma_columns(frame_f, ma_list)
    
    new_frame["MA_7_50"] = new_frame["MA_7"] - new_frame["MA_50"]
    new_frame["MA_14_100"] = new_frame["MA_14"] - new_frame["MA_100"]
    new_frame["MA_50_200"] = new_frame["MA_50"] - new_frame["MA_200"]
    
    frame = pd.read_pickle(stock_name)
    frame = frame[["HGDG_TARIH", "StockValue"]]
    new_frame = new_frame.join(frame)
    
    new_frame = buy_sell_recommendations(new_frame)
    new_frame = new_frame.rename(columns={"HGDG_TARIH": "Date"})
    
    loc_first = get_locations(new_frame)[0]
    loc_second = get_locations(new_frame)[1]
    
    first_transactions = get_transactions(new_frame, "rec_1", loc_first)
    second_transactions = get_transactions(new_frame, "rec_2", loc_second)

    first_returns = returns(new_frame, first_transactions)
    second_returns = returns(new_frame, second_transactions)
    
    first_cum_return = prod(first_returns)
    print(f"Final position of first transactions : {first_cum_return}")
    print(f"Duration (days) - total length       : {sum([t[1]-t[0] for t in first_transactions])} - {new_frame.shape[0]}")
    print(f"Returns of first transactions        : {[round(first,3) for first in first_returns]}")
    print(f"Locations of first transactions      : {first_transactions}")
    print("-"*85)
    
    second_cum_return  = prod(second_returns)
    print(f"Final position of second transactions: {second_cum_return}")
    print(f"Duration (days) - total length       : {sum([t[1]-t[0] for t in second_transactions])} - {new_frame.shape[0]}")
    print(f"Returns of second transactions       : {[round(second,3) for second in second_returns]}")
    print(f"Locations of second transactions     : {second_transactions}")
    print("-"*85)

    totalReturn = (first_cum_return + second_cum_return) / 2
    print(f"Ultimate position : {totalReturn}")
    
    return totalReturn

---

In [44]:
def get_only_frame_simulation(stock_name, start_date, end_date, ma_list, noise_first_par, noise_second_par):
    
    frame = pd.read_pickle(stock_name)
    start_date = convert_date(start_date)
    end_date = convert_date(end_date)
    frame = frame[(frame.HGDG_TARIH < end_date) & (frame.HGDG_TARIH > start_date)]
    
    frame_f = remove_noise(frame, noise_first_par, noise_second_par)
    new_frame = ma_columns(frame_f, ma_list)
    
    new_frame["MA_7_50"] = new_frame["MA_7"] - new_frame["MA_50"]
    new_frame["MA_14_100"] = new_frame["MA_14"] - new_frame["MA_100"]
    new_frame["MA_50_200"] = new_frame["MA_50"] - new_frame["MA_200"]
    
    frame = pd.read_pickle(stock_name)
    frame = frame[["HGDG_TARIH", "StockValue"]]
    new_frame = new_frame.join(frame)
    
    new_frame = buy_sell_recommendations(new_frame)
    new_frame = new_frame.rename(columns={"HGDG_TARIH": "Date"})
    
    return new_frame

In [45]:
def extract_and_store(stock_name, start_date, end_date):
    frame = sm.extract_stock_values(start_date, end_date, [stock_name])
    frame = frame.drop_duplicates(subset=["HGDG_HS_KODU", "HGDG_TARIH"], keep="last")
    frame = frame.rename(columns={"HGDG_KAPANIS": "StockValue"})
    frame = frame.sort_values(by=["HGDG_TARIH"])
    frame = frame.reset_index(drop=True)
    frame.to_pickle(stock_name)

In [ ]:
stock_name = "TKFEN"
ma_list=[7,14,50,100,200]
noise_first_par = 71
noise_second_par = 2

general_startDate = "31-01-2000"
general_endDate = "31-12-2020"

start_date = "01-01-2013"
first_end_date = "01-12-2017"

In [ ]:
# extract stock values and store
extract_and_store(stock_name, general_startDate, general_endDate)

In [ ]:
observations = []
for i in range(1000):
    frame = pd.read_pickle(stock_name)
    print(first_end_date)
    if frame[frame["HGDG_TARIH"] == first_end_date].shape[0] == 1:
        frame = get_only_frame_simulation(stock_name, start_date, first_end_date, ma_list, noise_first_par, noise_second_par)
        observations.append(frame.iloc[-1,:])
    else:
        observations = observations
    first_end_date = increment_date(first_end_date, 1)

In [ ]:
final_frame = pd.concat(observations, axis=1)

In [ ]:
final_frame = final_frame.T

In [ ]:
final_frame.to_pickle(f"{stock_name}TKFEN_Final_08_BUY_SELL")

In [136]:
final_frame = pd.read_pickle("TKFEN_Final_08_BUY_SELL")

In [137]:
final_frame.iloc[:,:]

,MA_7,MA_14,MA_50,MA_100,MA_200,MA_7_50,MA_14_100,MA_50_200,Date,StockValue,rec_1,rec_2
3162,16.6982,16.3036,14.6493,14.5384,16.2503,2.04892,1.76518,-1.60099,2020-06-30,17.02,BUY,BUY


In [138]:
final_frame.iloc[:,:]

,MA_7,MA_14,MA_50,MA_100,MA_200,MA_7_50,MA_14_100,MA_50_200,Date,StockValue,rec_1,rec_2
3162,16.6982,16.3036,14.6493,14.5384,16.2503,2.04892,1.76518,-1.60099,2020-06-30,17.02,BUY,BUY


In [110]:
get_locations(final_frame)

[[118, 232, 371], [123, 248, 260, 275, 387]]

In [111]:
first_transactions = get_transactions(final_frame, "rec_1", get_locations(final_frame)[0])
first_transactions

[[118, 232]]

In [112]:
second_transactions = get_transactions(final_frame, "rec_2", get_locations(final_frame)[1])
second_transactions

[[123, 248], [260, 275]]

In [113]:
first_returns = returns(final_frame, first_transactions)
first_returns

[0.13845094601726435]

In [114]:
second_returns = returns(final_frame, second_transactions)
second_returns

[0.17591422906925946, -0.029891272212091237]

In [115]:
first_cum_return = prod(first_returns)
second_cum_return  = prod(second_returns)

In [116]:
first_cum_return

1.1384509460172643

In [117]:
second_cum_return

1.1407646567500789

In [118]:
sum([t[1]-t[0] for t in first_transactions])

114

In [119]:
sum([t[1]-t[0] for t in second_transactions])

140